# Wet vegetation threshold tool

This notebook implements a widget for helping fihd good thresholds for wet vegetation.

In [6]:
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import xarray
from skimage.filters import threshold_otsu

In [11]:
def get_tcw(tile_label):
    tif_path_base = "/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_wetness/LS_TCW_PC_{}_1986_2020_summary.tif"
    tif_path = tif_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_rasterio(tif_path)
    return da

def get_mrvbf(tile_label):
    mrvbf_path_base = (
        mrvbf_base_path
    ) = "/g/data/u46/wofs/confidence_albers/MrVBF/tiles/mrvbf_{}.nc"
    mrvbf_path = mrvbf_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_dataset(mrvbf_path)
    return da

def get_tcg(tile_label):
    tif_path_base = "/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_{}_1986_2020_summary.tif"
    tif_path = tif_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_rasterio(tif_path)
    return da

In [12]:
%%bash
ls /g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/*.tif

/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-15_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-17_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-19_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-22_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-16_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-29_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_13_-27_1986_2020_summary.tif
/g/data/r78/datacube_stats/T

Unload all python modules, if you experience any issues.


In [33]:
%matplotlib widget
def show_widget(tile_label, subsample=5, continuous_update=True):
    tcw = get_tcw(tile_label).isel(band=3)[::subsample, ::subsample]
    tcg = get_tcg(tile_label).isel(band=1)[::subsample, ::subsample]
    vbf = get_mrvbf(tile_label).isel(time=0).band1[::subsample, ::subsample]
    
    tcw_mask = tcw > -600
    otsu_threshold = threshold_otsu(tcg.values[tcw_mask])
    initial_threshold = np.percentile(tcg, 95)
    tcg_mask = tcg < initial_threshold
    
    threshold_low = initial_threshold
    threshold_high = initial_threshold
    
    output = ipywidgets.Output()
    with output:
        fig = plt.figure(figsize=(11, 6))
        gs = fig.add_gridspec(ncols=2, nrows=2, width_ratios=[1, 1],
                              height_ratios=[1, 0.25], hspace=0.2, bottom=0.03, wspace=0.2,
                              top=0.95)
        ax = fig.add_subplot(gs[0, 0])
        im = (tcw_mask & tcg_mask).plot.imshow(interpolation='bilinear', ax=ax)
        hist_ax = fig.add_subplot(gs[1, :])
        hist = hist_ax.hist(tcg.values.ravel(), bins=100)
        
        hist_line_all = hist_ax.axvline(initial_threshold, c='red')
        hist_line_low = hist_ax.axvline(initial_threshold, c='magenta', alpha=0)
        hist_line_high = hist_ax.axvline(initial_threshold, c='purple', alpha=0)
        otsu_line = hist_ax.axvline(otsu_threshold, c='cyan')
        hist_ax.text(otsu_threshold, 5000, 'Otsu')
        
        vbf_ax = fig.add_subplot(gs[0, 1])
        vbf.plot.imshow(ax=vbf_ax, interpolation='gaussian')
    
    def slider_update_all(change):
        threshold = change['new']['value']
        thresholded = tcg < threshold
        nonlocal threshold_high, threshold_low
        threshold_high = threshold_low = threshold
        
        # Change the slider values without triggering the change event.
        slider_high.unobserve(slider_update_high)
        slider_low.unobserve(slider_update_low)
        slider_high.value = threshold_high
        slider_low.value = threshold_low
        slider_high.observe(slider_update_high)
        slider_low.observe(slider_update_low)
        
        with output:
            im.set_data(tcw_mask & thresholded)
            hist_line_all.set_alpha(1)
            hist_line_all.set_xdata([threshold, threshold])
            # Hide the high/low lines.
            hist_line_high.set_alpha(0)
            hist_line_low.set_alpha(0)
            im.autoscale()
            fig.canvas.draw()
    
    def slider_update_low(change):
        threshold = change['new']['value']
        nonlocal threshold_low
        threshold_low = threshold
        threshold_grid = np.where(vbf >= 3, threshold_high, threshold_low)
        thresholded = tcg < threshold_grid
        with output:
            im.set_data(tcw_mask & thresholded)
            # Hide the all line.
            hist_line_all.set_alpha(0)
            # Show the high/low lines.
            hist_line_high.set_alpha(1)
            hist_line_high.set_xdata([threshold_high, threshold_high])
            hist_line_low.set_alpha(1)
            hist_line_low.set_xdata([threshold_low, threshold_low])
            im.autoscale()
            fig.canvas.draw()
    
    def slider_update_high(change):
        threshold = change['new']['value']
        nonlocal threshold_high
        threshold_high = threshold
        threshold_grid = np.where(vbf >= 3, threshold_high, threshold_low)
        thresholded = tcg < threshold_grid
        with output:
            im.set_data(tcw_mask & thresholded)
            # Hide the all line.
            hist_line_all.set_alpha(0)
            # Show the high/low lines.
            hist_line_high.set_alpha(1)
            hist_line_high.set_xdata([threshold_high, threshold_high])
            hist_line_low.set_alpha(1)
            hist_line_low.set_xdata([threshold_low, threshold_low])
            im.autoscale()
            fig.canvas.draw()

    slider_props = dict(
        value=initial_threshold,
        min=tcg.min(),
        max=tcg.max(),
        step=10,
        disabled=False,
        continuous_update=continuous_update,
        orientation='horizontal',
        readout=True,
        style={'description_width': 'initial'},
        readout_format='.0f')
    slider_all = ipywidgets.FloatSlider(
        description='Max TCG50 (all):', 
        layout=ipywidgets.Layout(width='90%', margin='0 0 0 0'), **slider_props)
    slider_all.observe(slider_update_all)
    slider_low = ipywidgets.FloatSlider(
        description='Max TCG50 (MrVBF < 3):',
        layout=ipywidgets.Layout(width='100%', margin='0 0 0 0'),
        **slider_props)
    slider_low.observe(slider_update_low)
    slider_high = ipywidgets.FloatSlider(
        description='Max TCG50 (MrVBF >= 3):',
        layout=ipywidgets.Layout(width='100%', margin='0 0 0 0'),
        **slider_props)
    slider_high.observe(slider_update_high)

    return ipywidgets.VBox([
        output, slider_all,
        ipywidgets.HBox([slider_low, slider_high], layout=ipywidgets.Layout(width='90%', margin='0 0 0 0'))], layout=ipywidgets.Layout(margin='0 0 0 0'))

In [34]:
show_widget('14,-21', subsample=5, continuous_update=True)